In [110]:
import pandas as pd
import requests
import numpy as np

data = {
    "2022": pd.read_csv("~/code/brentfordfc/results/data/2021_2022.csv"),
    "2023": pd.read_csv("~/code/brentfordfc/results/data/2022_2023.csv")
}

In [78]:
def process_data(data, year):
    points_map = {
        "W": 3,
        "D": 1,
        "L": 0
    }

    relegation_promotion_map = {
        "2022": {
            "Burnley": "Burnley/Fulham",
            "Watford": "Watford/Bournemouth",
            "Norwich City": "Norwich City/Nott'ham Forest"
        },
        "2023": {
            "Fulham": "Burnley/Fulham",
            "Bournemouth": "Watford/Bournemouth",
            "Nott'ham Forest": "Norwich City/Nott'ham Forest"
        }
    }

    df = (
        data
        .loc[lambda df: df["Comp"] == "Premier League"]
        .assign(Points=lambda df: df["Result"].map(points_map))
        .assign(Opponent=lambda df: df["Opponent"].map(relegation_promotion_map[year]).fillna(df["Opponent"]))
        .assign(dxG=lambda df: df["xG"] - df["xGA"])
        .assign(xGwin=lambda df: df["dxG"] > 0)
        .assign(Round=lambda df: df["Round"].str.replace("Matchweek ", "").astype(int))
        [["Opponent", "Venue", "Round", "GF", "GA", "Result", "Points", "xG", "xGA", "dxG", "xGwin"]]
    )

    df.columns = [f"{col}_{year}" if col not in {"Opponent", "Venue"} else col for col in df.columns ]
    return df

In [79]:
curated = {year: process_data(data[year], year) for year in data.keys()}

In [80]:
df = (
    curated["2022"]
    .merge(curated["2023"], how="outer", on=["Opponent", "Venue"])
    .sort_values("Round_2023")
)

df.head()

,Opponent,Venue,Round_2022,GF_2022,GA_2022,Result_2022,Points_2022,xG_2022,xGA_2022,dxG_2022,xGwin_2022,Round_2023,GF_2023,GA_2023,Result_2023,Points_2023,xG_2023,xGA_2023,dxG_2023,xGwin_2023
29,Leicester City,Away,30,1,2,L,0,1.3,1.0,0.3,True,1,2,2,D,1.0,0.8,0.6,0.2,True
21,Manchester Utd,Home,17,1,3,L,0,2.3,2.3,0.0,False,2,4,0,W,3.0,1.6,0.9,0.7,True
9,Burnley/Fulham,Away,10,1,3,L,0,0.8,2.3,-1.5,False,3,2,3,L,0.0,1.8,2.4,-0.6,False
12,Everton,Home,13,1,0,W,3,1.2,1.0,0.2,True,4,1,1,D,1.0,2.1,1.1,1.0,True
1,Crystal Palace,Away,2,0,0,D,1,0.9,0.5,0.4,True,5,1,1,D,1.0,1.4,1.0,0.4,True


In [ ]:
pd.Series.

In [115]:
difference = (
    df
    [["Opponent", "Venue", "Points_2022", "Points_2023"]]
    .assign(difference_202223=lambda df: df["Points_2023"] - df["Points_2022"])
    .assign(cumratio=lambda df: df["Points_2023"].cumsum() / df["Points_2022"].cumsum())
    .reset_index(drop=True)
)

latest_ratio = difference["cumratio"].dropna().iloc[-1]
avg_by_result = (
    difference
    .groupby("Points_2022")
    ["Points_2023"]
    .mean()
    .reset_index()
    .rename({"Points_2023": "avg_by_result"}, axis=1)
)

difference = difference.merge(avg_by_result, how="left", on=["Points_2022"])

In [116]:
difference["projected_points"] = np.select(
    [difference["Points_2023"].notna()],
    [difference["Points_2023"]],
    difference["avg_by_result"]
)

In [122]:
print(f'Current actual difference between 2022 and 2023: {difference["difference_202223"].sum()}')
print(f'Latest cumulative ratio between 2022 and 2023: {latest_ratio: .2f}')
print(f'Projected points for 2023: {difference["projected_points"].sum(): .2f}')

Current actual difference between 2022 and 2023: 10.0
Latest cumulative ratio between 2022 and 2023:  1.22
Projected points for 2023:  57.59
